In [ ]:
from scrdet_parallel import Detector
import time # not required, only used for this notebook

# 1) Initialize the detector with GPU #2 and GPU #0 (assumes you have at least 3 GPU's accessible )

In [ ]:
start = time.time()
detector = Detector([2,0])
end = time.time()
print('Elapsed time = ',end-start)

Initializing detector and warming up 2 GPU(s)...
Initialization complete. Ready to run predict.
Elapsed time =  56.8215970993042


In [ ]:
file = 'Sample_NITF/imagename.r0.ntf'

# 2a) Run detector on a NITF with some pre-determined NITF clips.
# Note that the clip format is clip=[[left0,bottom0,right0,top0],...[leftn,bottomn,rightn,topn]]

In [ ]:
start = time.time()
clips = [[11000, 11000, 14000, 8000],[200, 12500, 3000, 10000],[15000, 17000, 17000, 15000]]
predictions_clip = detector.predict(file=file,clips=clips)
end = time.time()
print('Elapsed time = ',end-start)

Runnning a single image
Completed Image 1 of 1                          
Elapsed time =  9.681691408157349


# 2b) Alternatively, run detector on the whole NITF

In [ ]:
start = time.time()
predictions_full = detector.predict(file=file)
end = time.time()
print('Elapsed time = ',end-start)

Runnning a single image
Completed Image 1 of 1                              
Elapsed time =  160.57559514045715


# 3) Verify the output format

In [ ]:
print(type(predictions_clip))
predictions_clip.head()

NameError: name 'predictions_clip' is not defined

# 4) Format results for the evaluator module and write the results to a .txt file for later evaluation

In [ ]:
def format_predictions(predictions):
    '''formats the pandas dataframe .precict output
       to the .txt format used by the evaluator
       module
    '''
    # Turn the Polygon coordinates into individual columns
    x0 = [];y0 = [];x1= [];y1 = [];x2 = [];y2 = [];x3 = [];y3 = []
    for index, row in predictions.iterrows():
        x0.append(row['geometry'].exterior.xy[0][0])
        y0.append(row['geometry'].exterior.xy[1][0])
        x1.append(row['geometry'].exterior.xy[0][1])
        y1.append(row['geometry'].exterior.xy[1][1])
        x2.append(row['geometry'].exterior.xy[0][2])
        y2.append(row['geometry'].exterior.xy[1][2])
        x3.append(row['geometry'].exterior.xy[0][3])
        y3.append(row['geometry'].exterior.xy[1][3])
    predictions['x0'] = x0
    predictions['y0'] = y0
    predictions['x1'] = x1
    predictions['y1'] = y1
    predictions['x2'] = x2
    predictions['y2'] = y2
    predictions['x3'] = x3
    predictions['y3'] = y3
    return predictions

In [ ]:
# format the results and write to a text file
formatted_predictions = format_predictions(predictions_clip)
formatted_predictions.to_csv('Test&Evaluate/my_results.txt',sep=' ',index=False,header=False,columns=['class','image_name','conf','x0','y0',
                                                              'x1','y1','x2','y2','x3','y3','Background',
                                                               'class1','class2','class3'])

# 5) Plot the whole NITF with prediction results from the clip run

In [ ]:
## This cell imports tools for reading and plotting
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib import cm, colors
from osgeo.gdal import Open as gdalOpen
import cv2
import numpy as np

In [ ]:
## This cell reads and adjusts an image

##################
#### READ PNG ####
##################
#img = cv2.imread(file)
######################
######################

##################
### READ NITF ####
##################
my_nitf = gdalOpen(file)
text = 'Reading Image...'
print(text,end="\r")
img = my_nitf.GetRasterBand(1).ReadAsArray()
#################################################
#### optionally adjust nitf visibility ##########
from scrdet_parallel import adjust_image ########
bits = int(my_nitf.GetMetadata()['NITF_ABPP'])###
img = adjust_image(img,bit_depth_in=bits) #######
#################################################
#################################################
print(' '*len(text),end="\r")
print('Done')
##########################
##########################

Done            


In [ ]:
## This cell plots everything
fig1,ax = plt.subplots(figsize=(100,100))
ax.imshow(img,cmap='gray');
classes = np.array(sorted(predictions_clip['class'].unique()))
colors = cm.get_cmap('rainbow',len(classes))

# plot the detections
for index, row in predictions_clip[predictions_clip['conf'] > 0.8].iterrows():
    cli = np.where(classes == row['class'])[0][0]
    ax.plot(*row['geometry'].exterior.xy,c=colors(cli))
    
# plot the clips    
for clip in clips:
    ax.plot([clip[0],clip[2],clip[2],clip[0],clip[0]],[clip[3],clip[3],clip[1],clip[1],clip[3]],'r',linewidth=4)

legend_elements = []
for c in classes:
    cli = np.where(classes == c)[0][0]
    legend_elements.append(Patch(color=colors(cli),label=c))
ax.legend(handles=legend_elements,loc='upper left',fontsize=60);

NameError: name 'plt' is not defined